In [1]:
import rasterio

# Path to your clipped LULC file
lulc_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"
with rasterio.open(lulc_path) as src:
    bounds = src.bounds
    crs = src.crs
    width = src.width
    height = src.height
    res = src.res

print("CRS:", crs)
print("Bounding Box:")
print(f"  Min Lon: {bounds.left}")
print(f"  Min Lat: {bounds.bottom}")
print(f"  Max Lon: {bounds.right}")
print(f"  Max Lat: {bounds.top}")
print("Resolution (Lon, Lat):", res)
print(f"Pixel Dimensions: {width} x {height}")


CRS: EPSG:32644
Bounding Box:
  Min Lon: 108365.73527523666
  Min Lat: 3152714.2869458776
  Max Lon: 499835.73527523666
  Max Lat: 3491954.2869458776
Resolution (Lon, Lat): (30.0, 30.0)
Pixel Dimensions: 13049 x 11308


In [2]:
import rasterio
from rasterio.windows import from_bounds

# Path to your LULC file
lulc_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"

# Geographic bounds for N31_E80 tile
# Covers 31°N to 32°N latitude and 80°E to 81°E longitude
tile_min_lon, tile_min_lat = 80.0, 31.0
tile_max_lon, tile_max_lat = 81.0, 32.0

with rasterio.open(lulc_path) as src:
    # Reproject bounds to LULC CRS if needed
    if src.crs.to_string() != "EPSG:4326":
        from pyproj import Transformer
        transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
        tile_min_lon, tile_min_lat = transformer.transform(tile_min_lon, tile_min_lat)
        tile_max_lon, tile_max_lat = transformer.transform(tile_max_lon, tile_max_lat)

    # Clip window for the tile area
    try:
        window = from_bounds(tile_min_lon, tile_min_lat, tile_max_lon, tile_max_lat, src.transform)
    except ValueError:
        print("Tile is completely outside LULC raster.")
        exit()

    # Read data for that window
    data = src.read(1, window=window, masked=True)

    # Check if there are any valid pixels (not nodata)
    if data.mask.all():
        print("✅ No LULC coverage in N31_E80 — safe to skip DEM tile.")
    else:
        print("⚠ LULC coverage found in N31_E80 — DEM tile needed.")


⚠ LULC coverage found in N31_E80 — DEM tile needed.


In [2]:
import rasterio
from rasterio.transform import from_origin
import numpy as np

# Input .hgt file
hgt_file = r"C:\Users\Ankit\Downloads\N31E081.SRTMGL1.hgt\N31E081.hgt"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets\N31E080.tif"

# Known lat/lon for this tile
lat = 31
lon = 80

# SRTM tiles are 3601x3601 with 1 arc-second resolution (~30m)
size = 3601

# Read binary elevation data
with open(hgt_file, "rb") as f:
    elevations = np.fromfile(f, np.dtype('>i2'), size * size).reshape((size, size))

# Transform for GeoTIFF
transform = from_origin(lon, lat + 1, 1/3600, 1/3600)  # 1 arc-second resolution

# Save as GeoTIFF
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=elevations.shape[0],
    width=elevations.shape[1],
    count=1,
    dtype=elevations.dtype,
    crs='EPSG:4326',
    transform=transform
) as dst:
    dst.write(elevations, 1)

print(f"✅ Converted {hgt_file} to {output_tif}")

TypeError: invalid dtype: dtype('>i2')

In [3]:
import rasterio
from rasterio.transform import from_origin
import numpy as np

# Input .hgt file
hgt_file = r"C:\Users\Ankit\Downloads\N31E081.SRTMGL1.hgt\N31E081.hgt"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets\N31E080.tif"


# Known lat/lon for this tile
lat = 31
lon = 80

# SRTM tiles are 3601x3601 with 1 arc-second resolution (~30m)
size = 3601

# Read binary elevation data (big-endian) and convert to native endianness
with open(hgt_file, "rb") as f:
    elevations = np.fromfile(f, np.dtype('>i2'), size * size).reshape((size, size)).astype(np.int16)

# Transform for GeoTIFF
transform = from_origin(lon, lat + 1, 1/3600, 1/3600)  # 1 arc-second resolution

# Save as GeoTIFF
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=elevations.shape[0],
    width=elevations.shape[1],
    count=1,
    dtype='int16',  # native dtype string
    crs='EPSG:4326',
    transform=transform
) as dst:
    dst.write(elevations, 1)

print(f"✅ Converted {hgt_file} to {output_tif}")


✅ Converted C:\Users\Ankit\Downloads\N31E081.SRTMGL1.hgt\N31E081.hgt to C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets\N31E080.tif


In [4]:
import rasterio
from rasterio.merge import merge
import glob
import os

# Path to your DEM tiles folder
dem_folder = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets"

# Output merged DEM file
output_merged = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\merged_DEM.tif"

# Get all DEM files (.hgt or .tif)
# If they are still .hgt files, convert to .tif first — this script assumes .tif
dem_files = glob.glob(os.path.join(dem_folder, "*.tif"))

# Read and store datasets
src_files_to_mosaic = []
for fp in dem_files:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# Merge DEM tiles
mosaic, out_transform = merge(src_files_to_mosaic)

# Copy the metadata
out_meta = src.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform,
    "crs": src.crs
})

# Save merged DEM
with rasterio.open(output_merged, "w", **out_meta) as dest:
    dest.write(mosaic)

print(f"✅ Merged DEM saved to: {output_merged}")


✅ Merged DEM saved to: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\merged_DEM.tif


In [7]:
import rasterio
from shapely.geometry import box
import math

# ==== INPUT FILES ====
lulc_file = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"   # or .shp
dem_file  = r"C:\Users\Ankit\Downloads\merged_DEM.tif"

# Read DEM bounds
with rasterio.open(dem_file) as dem:
    dem_bounds = dem.bounds  # (minx, miny, maxx, maxy)
    dem_crs = dem.crs

# Read LULC bounds
with rasterio.open(lulc_file) as lulc:
    lulc_bounds = lulc.bounds
    lulc_crs = lulc.crs

# Ensure both are in same CRS (EPSG:4326)
if dem_crs != lulc_crs:
    raise ValueError("CRS mismatch! Reproject one of the datasets to EPSG:4326 before running.")

# Convert to shapely polygons for comparison
dem_poly = box(*dem_bounds)
lulc_poly = box(*lulc_bounds)

# Check coverage
if dem_poly.contains(lulc_poly):
    print("✅ DEM fully covers LULC extent — No new tiles needed.")
else:
    print("⚠ DEM does NOT fully cover LULC extent.")
    missing_area = lulc_poly.difference(dem_poly)
    print(f"Missing area bounds: {missing_area.bounds}")

    # Calculate missing tiles in Nxx_Exxx format
    minx, miny, maxx, maxy = missing_area.bounds
    min_lat = math.floor(miny)
    max_lat = math.floor(maxy)
    min_lon = math.floor(minx)
    max_lon = math.floor(maxx)

    missing_tiles = []
    for lat in range(min_lat, max_lat + 1):
        for lon in range(min_lon, max_lon + 1):
            ns = "N" if lat >= 0 else "S"
            ew = "E" if lon >= 0 else "W"
            missing_tiles.append(f"{ns}{abs(lat):02d}{ew}{abs(lon):03d}")

    print("📌 Missing DEM tiles you need to download:")
    for t in missing_tiles:
        print(t)


ValueError: CRS mismatch! Reproject one of the datasets to EPSG:4326 before running.

In [9]:
import rasterio
from shapely.geometry import box, mapping
import math
import geopandas as gpd

# ==== INPUT FILES ====
lulc_file = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"   # or .shp
dem_file  = r"C:\Users\Ankit\Downloads\merged_DEM.tif"

# Read DEM
with rasterio.open(dem_file) as dem:
    dem_bounds = dem.bounds
    dem_crs = dem.crs

# Read LULC
with rasterio.open(lulc_file) as lulc:
    lulc_bounds = lulc.bounds
    lulc_crs = lulc.crs

# If CRSs differ, reproject LULC bounds to DEM CRS for coverage check
if dem_crs != lulc_crs:
    print("CRS mismatch — reprojecting LULC bounds to DEM CRS for coverage check...")
    lulc_poly = gpd.GeoSeries([box(*lulc_bounds)], crs=lulc_crs).to_crs(dem_crs).iloc[0]
else:
    lulc_poly = box(*lulc_bounds)

dem_poly = box(*dem_bounds)

# Check coverage
if dem_poly.contains(lulc_poly):
    print("✅ DEM fully covers LULC extent — No new tiles needed.")
else:
    print("⚠ DEM does NOT fully cover LULC extent.")
    missing_area = lulc_poly.difference(dem_poly)
    print(f"Missing area bounds (in DEM CRS): {missing_area.bounds}")

    # Reproject missing area to EPSG:4326 to get tile coordinates
    missing_area_wgs = gpd.GeoSeries([missing_area], crs=dem_crs).to_crs("EPSG:4326").iloc[0]
    minx, miny, maxx, maxy = missing_area_wgs.bounds

    # Calculate missing tiles in NxxExxx format
    min_lat = math.floor(miny)
    max_lat = math.floor(maxy)
    min_lon = math.floor(minx)
    max_lon = math.floor(maxx)

    missing_tiles = []
    for lat in range(min_lat, max_lat + 1):
        for lon in range(min_lon, max_lon + 1):
            ns = "N" if lat >= 0 else "S"
            ew = "E" if lon >= 0 else "W"
            missing_tiles.append(f"{ns}{abs(lat):02d}{ew}{abs(lon):03d}")

    print("📌 Missing DEM tiles you need to download:")
    for t in missing_tiles:
        print(t)


CRS mismatch — reprojecting LULC bounds to DEM CRS for coverage check...
⚠ DEM does NOT fully cover LULC extent.
Missing area bounds (in DEM CRS): (76.87778249225309, 28.495715625381205, 77.0000003622, 31.498087210948626)
📌 Missing DEM tiles you need to download:
N28E076
N28E077
N29E076
N29E077
N30E076
N30E077
N31E076
N31E077


In [10]:
import rasterio
import numpy as np
from scipy import ndimage

# Path to your merged DEM
merged_dem_path = r"C:\Users\Ankit\Downloads\merged_DEM.tif"
filled_dem_path  = r"C:\Users\Ankit\Downloads\merged_DEM_filled.tif"

# Step 1: Open merged DEM
with rasterio.open(merged_dem_path) as src:
    dem_data = src.read(1)  # first band
    profile = src.profile
    nodata_value = src.nodata

# Step 2: Detect nodata mask
if nodata_value is None:
    # If no explicit nodata, assume negative or extreme values are invalid
    nodata_value = -9999

mask = (dem_data == nodata_value) | np.isnan(dem_data)
missing_count = np.sum(mask)
print(f"🔍 Missing pixels found: {missing_count}")

# Step 3: Fill missing values by interpolation (IDW via scipy)
if missing_count > 0:
    print("🛠 Filling missing values...")
    
    # Create coordinate arrays
    coords = np.array(np.nonzero(~mask)).T
    values = dem_data[~mask]
    
    # Distance transform gives nearest non-missing pixel index
    filled_data = dem_data.copy()
    nearest_index = ndimage.distance_transform_edt(mask, return_distances=False, return_indices=True)
    filled_data[mask] = dem_data[tuple(nearest_index[:, mask])]
    
    # Save filled DEM
    with rasterio.open(filled_dem_path, 'w', **profile) as dst:
        dst.write(filled_data, 1)
    
    print(f"✅ Missing values filled. Saved as: {filled_dem_path}")
else:
    print("✅ No missing values found — DEM is complete.")


🔍 Missing pixels found: 0
✅ No missing values found — DEM is complete.


In [12]:
import os, re, math
import rasterio
from pyproj import Transformer

# ====== EDIT THESE ======
lulc_file      = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"  # your LULC raster
dem_tiles_dir  = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets"       # folder with all .tif/.hgt tiles (NRSC + SRTM)
merged_dem_file = r"C:\Users\Ankit\Downloads\merged_DEM.tif"             # optional: your merged DEM (used only for reporting)
# ========================

def to_wgs84_bounds(path):
    """Return (minlon, minlat, maxlon, maxlat) for a raster, reprojected to EPSG:4326."""
    with rasterio.open(path) as src:
        minx, miny, maxx, maxy = src.bounds
        src_crs = src.crs
    if src_crs is None:
        raise ValueError(f"{path} has no CRS.")
    if str(src_crs).upper() in ("EPSG:4326", "WGS84", "EPSG:4326+WGS84"):
        return (minx, miny, maxx, maxy)
    tr = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
    ll = tr.transform(minx, miny)
    ur = tr.transform(maxx, maxy)
    # ensure proper ordering
    minlon, minlat = min(ll[0], ur[0]), min(ll[1], ur[1])
    maxlon, maxlat = max(ll[0], ur[0]), max(ll[1], ur[1])
    return (minlon, minlat, maxlon, maxlat)

def code_from_latlon(lat, lon):
    ns = "N" if lat >= 0 else "S"
    ew = "E" if lon >= 0 else "W"
    return f"{ns}{abs(lat):


SyntaxError: unterminated string literal (detected at line 31) (138410276.py, line 31)

In [3]:
import rasterio
with open(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif") as src:
    ds = src.read

In [11]:
ds

AttributeError: 'builtin_function_or_method' object has no attribute 'show'